In [6]:
import os
import re
import time
import random
import logging
import numpy as np
import pandas as pd
import requests

import spacy
import PyPDF2
import docx
from PIL import Image
import pytesseract
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

###############################################
# Part 1: Shared Web Scraping Functions
###############################################
def scroll_to_load_all(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def is_valid_link(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

###############################################
# Part 2: Zscaler Job Scraping
###############################################
def scrape_zscaler_jobs(job_title_filter):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    jobs_data = []
    base_url = "https://boards.greenhouse.io/zscaler"
    company_name = "Zscaler"

    try:
        driver.get(base_url)
        print(f"Scraping Zscaler jobs, filtering for '{job_title_filter}'...")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "opening")))
        scroll_to_load_all(driver)
        job_elements = driver.find_elements(By.CLASS_NAME, "opening")
        for job in job_elements:
            try:
                a_elem = job.find_element(By.CSS_SELECTOR, "a")
                title = a_elem.text.strip()
                link = a_elem.get_attribute("href")
                locations = job.find_elements(By.CLASS_NAME, "location")
                location = locations[0].text.strip() if locations else "Not specified"
                if job_title_filter.lower() in title.lower():
                    if is_valid_link(link):
                        jobs_data.append({
                            "Title": title,
                            "Location": location,
                            "Link": link,
                            "Company": company_name
                        })
                    else:
                        print(f"Skipping job '{title}' due to invalid link.")
            except Exception as e:
                print(f"Error parsing job from Zscaler: {e}")
                continue
        print(f"Found {len(jobs_data)} jobs matching '{job_title_filter}' from Zscaler.")
    except Exception as e:
        print(f"Error during scraping from Zscaler: {e}")
    finally:
        driver.quit()
    return jobs_data

###############################################
# Part 3: Amazon Job Scraping
###############################################
def scrape_amazon_jobs(job_title):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    jobs_data = []
    
    base_url = "https://www.amazon.jobs/en/search?base_query={}"
    search_url = base_url.format(job_title.replace(" ", "+"))
    job_container = "job-tile"
    title_class = "job-title"
    location_class = "location-and-id"
    company_name = "Amazon"

    try:
        driver.get(search_url)
        print(f"Scraping Amazon jobs, filtering for '{job_title}'...")
        try:
            WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
            ).click()
            print("Accepted Amazon popup.")
        except:
            print("No accept/reject popup found for Amazon.")
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, job_container)))
        scroll_to_load_all(driver)
        job_elements = driver.find_elements(By.CLASS_NAME, job_container)
        
        for job in job_elements:
            try:
                title = job.find_element(By.CLASS_NAME, title_class).text.strip()
                link = job.find_element(By.TAG_NAME, "a").get_attribute("href")
                location = job.find_element(By.CLASS_NAME, location_class).text.strip() if job.find_elements(By.CLASS_NAME, location_class) else "Not specified"
                if job_title.lower() in title.lower():
                    if is_valid_link(link):
                        jobs_data.append({
                            "Title": title,
                            "Location": location,
                            "Link": link,
                            "Company": company_name
                        })
                    else:
                        print(f"Skipping job '{title}' due to invalid link.")
            except Exception as e:
                print(f"Error parsing job from Amazon: {e}")
                continue
        print(f"Found {len(jobs_data)} jobs matching '{job_title}' from Amazon.")
    except Exception as e:
        print(f"Error during scraping from Amazon: {e}")
    finally:
        driver.quit()
    return jobs_data

###############################################
# Part 4: LinkedIn Job Scraping (No Login)
###############################################
def scrape_linkedin_jobs(job_title, num_pages=3):
    # Build URL based solely on the job title (no location filter)
    job_title_formatted = job_title.replace(" ", "%20")
    url = f"https://www.linkedin.com/jobs/search?keywords={job_title_formatted}"
    
    # Setup Chrome options for LinkedIn scraping
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Update User-Agent to appear as a regular browser
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    })
    
    jobs_data = []
    try:
        driver.get(url)
        time.sleep(3)  # Allow page to load
        
        for page in range(num_pages):
            print(f"LinkedIn: Scraping page {page+1}...")
            try:
                job_list = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "jobs-search__results-list"))
                )
                job_cards = job_list.find_elements(By.CSS_SELECTOR, ".jobs-search__results-list > li")
                print(f"Found {len(job_cards)} job cards on page {page+1}")
                
                for i, job in enumerate(job_cards):
                    try:
                        try:
                            title_element = job.find_element(By.CSS_SELECTOR, "h3.base-search-card__title")
                            title = title_element.text.strip()
                        except:
                            title = "N/A"
                        
                        try:
                            company_element = job.find_element(By.CSS_SELECTOR, "h4.base-search-card__subtitle")
                            company = company_element.text.strip()
                        except:
                            company = "N/A"
                        
                        try:
                            # Attempt to get location (if available)
                            location_element = job.find_element(By.CSS_SELECTOR, ".job-search-card__location")
                            location = location_element.text.strip()
                        except:
                            location = "Not specified"
                        
                        try:
                            job_link_element = job.find_element(By.CSS_SELECTOR, "a.base-card__full-link")
                            link = job_link_element.get_attribute("href")
                        except:
                            link = "N/A"
                        
                        job_description = "N/A"
                        try:
                            # Click the job card to load details
                            title_element.click()
                            time.sleep(random.uniform(1, 2))
                            desc_element = WebDriverWait(driver, 5).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, ".show-more-less-html__markup"))
                            )
                            job_description = desc_element.text.strip()
                        except:
                            job_description = "Login required to view full description"
                        
                        job_data = {
                            "Title": title,
                            "Company": company,
                            "Location": location,
                            "Link": link,
                            "Description": job_description
                        }
                        jobs_data.append(job_data)
                        print(f"LinkedIn: Scraped job {i+1}/{len(job_cards)}: {title} at {company}")
                    except Exception as e:
                        print(f"LinkedIn: Error scraping job {i+1}: {str(e)}")
                    time.sleep(random.uniform(0.5, 1.5))
            except Exception as e:
                print(f"LinkedIn: Error on page {page+1}: {str(e)}")
            
            if page < num_pages - 1:
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(1)
                    next_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label='Next']"))
                    )
                    next_button.click()
                    print("LinkedIn: Navigating to next page...")
                    time.sleep(random.uniform(3, 5))
                except TimeoutException:
                    print("LinkedIn: No more pages available or login required for pagination")
                    break
    except Exception as e:
        print(f"LinkedIn: Error during scraping: {str(e)}")
    finally:
        driver.quit()
    
    print(f"LinkedIn: Found {len(jobs_data)} jobs matching '{job_title}'")
    return jobs_data

###############################################
# Part 5: Enrich Job Listings for Matching
###############################################
def enrich_job_listing(job):
    
    enriched = {}
    enriched['title'] = job.get("Title", "")
    enriched['company'] = job.get("Company", "")
    enriched['location'] = job.get("Location", "")
    # For LinkedIn, the key might be "Link" already (or "Job Link"); we assume "Link"
    enriched['link'] = job.get("Link", job.get("Job Link", ""))
    title_lower = enriched['title'].lower()
    if "engineer" in title_lower or "developer" in title_lower:
        enriched['domain'] = "Engineering"
    elif "security" in title_lower:
        enriched['domain'] = "Cybersecurity"
    elif "sales" in title_lower:
        enriched['domain'] = "Sales"
    else:
        enriched['domain'] = "General"
    enriched['salary_range'] = "Not specified"
    enriched['experience'] = "0-2"  # Default experience range (min-max)
    enriched['skills'] = []       # Can be enriched if job description is available
    enriched['certifications_required'] = []  # Can be updated based on job details
    return enriched

###############################################
# Part 6: Advanced Resume Recommendation System
###############################################
class BatchResumeRecommendationSystem:
    def __init__(self, resume_folder, log_file='resume_recommendations.log'):
        logging.basicConfig(filename=log_file, level=logging.INFO, 
                            format='%(asctime)s - %(message)s')
        self.nlp = spacy.load('en_core_web_sm')
        self.resume_folder = resume_folder
        self.input_shape = None  
        self.model = self.build_model()  # Model will be rebuilt once input shape is known
        
        self.domain_keywords = {
            'Engineering': ['engineer', 'developer', 'software', 'technical'],
            'Cybersecurity': ['security', 'cyber', 'threat', 'risk'],
            'Sales': ['sales', 'business development'],
            'Marketing': ['marketing', 'seo', 'content'],
            'Operations': ['operations', 'ops']
        }
        self.job_listings = []  # To be set externally

    def extract_text_from_resume(self, file_path):
        try:
            file_extension = os.path.splitext(file_path)[1].lower()
            if file_extension == '.pdf':
                with open(file_path, 'rb') as file:
                    reader = PyPDF2.PdfReader(file)
                    text = ' '.join([page.extract_text() for page in reader.pages if page.extract_text()])
            elif file_extension == '.docx':
                doc = docx.Document(file_path)
                text = ' '.join([paragraph.text for paragraph in doc.paragraphs])
            elif file_extension == '.txt':
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
            elif file_extension in ['.png', '.jpg', '.jpeg']:
                image = Image.open(file_path)
                text = pytesseract.image_to_string(image)
            else:
                logging.warning(f"Unsupported file type: {file_extension} for {file_path}")
                return None
            return text
        except Exception as e:
            logging.error(f"Error extracting text from {file_path}: {str(e)}")
            return None

    def extract_resume_features(self, resume_text):
        resume_text = resume_text.lower()
        doc = self.nlp(resume_text)
        features = {
            'skills': [],
            'education_level': None,
            'experience_years': 0,
            'domains': [],
            'project_keywords': [],
            'certifications': []
        }
        skill_patterns = [
            'python', 'java', 'c++', 'machine learning', 'data analysis', 'aws', 'azure', 'sql', 'project management',
            'tensorflow', 'pytorch', 'cloud computing', 'spring', 'hibernate', 'autocad', 'revit', 'gis', 'r',
            'statistics', 'excel', 'financial modeling', 'docker', 'devops', 'microservices', 'kubernetes', 'terraform',
            'cybersecurity', 'network administration', 'sagemaker',
            'javascript', 'html', 'css', 'react', 'node.js', 'flask', 'django', 'php', 'sql server', 'mongodb',
            'c#', 'ruby', 'spark', 'hadoop', 'kotlin', 'swift', 'go', 'laravel', 'express.js'
        ]
        features['skills'] = [skill for skill in skill_patterns if skill in resume_text]
        
        education_keywords = {
            'PhD': ['phd', 'doctorate'],
            'Masters': ['master', 'ms', 'ma'],
            'Bachelors': ['bachelor', 'bs', 'ba']
        }
        for level, keywords in education_keywords.items():
            if any(keyword in resume_text for keyword in keywords):
                features['education_level'] = level
                break
        
        experience_match = re.search(r'(\d+)\s*(?:years?|yrs?)\s*(?:of)?\s*experience', resume_text, re.IGNORECASE)
        if experience_match:
            features['experience_years'] = int(experience_match.group(1))
        
        for domain, keywords in self.domain_keywords.items():
            if any(keyword in resume_text for keyword in keywords):
                features['domains'].append(domain)
        
        project_keywords = re.findall(r'project\s*:\s*([^\n]+)', resume_text, re.IGNORECASE)
        features['project_keywords'] = project_keywords
        
        certification_keywords = [
            'aws certified', 'cissp', 'oracle certified', 'microsoft certified', 'comptia', 'cisco certified',
            'tensorflow developer', 'pytorch expert', 'certified data scientist', 'leed accredited',
            'certified sustainability professional', 'java certified', 'pe license', 'civil engineering certification',
            'healthcare analytics certified', 'cfa', 'cpa', 'scrum master','snow pro certification',"compTIA a+",
            "comptia network+",
            "comptia security+",
            "comptia server+",
            "comptia cloud+",
            "cisco ccna",
            "cisco ccnp",
            "cisco ccie",
            "microsoft mcsa",
            "microsoft mcse",
            "microsoft certified: azure fundamentals/administrator",
            "cissp",
            "cism",
            "certified ethical hacker (ceh)",
            "red hat certified system administrator (rhcsa)",
            "red hat certified engineer (rhce)",
            "ibm data science professional certificate",
            "dasca associate", "data scientist",
            "coursera data science specialization", "sas certified data scientist",
            "microsoft certified: azure data scientist associate","google professional machine learning engineer",
            "aws certified machine learning – specialty",
            "microsoft certified: azure ai engineer associate",
            "tensorflow developer certificate"
        ]
        features['certifications'] = [cert for cert in certification_keywords if cert in resume_text]
        
        return features

    def vectorize_features(self, resume_features, job_listing):
        all_skills = set(resume_features['skills'] + job_listing['skills'])
        resume_skills_vec = [1 if skill in resume_features['skills'] else 0 for skill in all_skills]
        job_skills_vec = [1 if skill in job_listing['skills'] else 0 for skill in all_skills]
        
        all_certs = set(resume_features['certifications'] + job_listing['certifications_required'])
        resume_certs_vec = [1 if cert in resume_features['certifications'] else 0 for cert in all_certs]
        job_certs_vec = [1 if cert in job_listing['certifications_required'] else 0 for cert in all_certs]
        
        resume_exp = resume_features['experience_years']
        try:
            job_exp_min = int(job_listing['experience'].split('-')[0])
        except:
            job_exp_min = 0
        exp_diff = resume_exp - job_exp_min
        
        domain_match = 1 if any(domain.lower() in job_listing['domain'].lower() for domain in resume_features['domains']) else 0
        
        feature_vector = resume_skills_vec + job_skills_vec + resume_certs_vec + job_certs_vec + [exp_diff, domain_match]
        
        if self.input_shape is None:
            self.input_shape = len(feature_vector)
            self.model = self.build_model()
        return np.array(feature_vector)

    def build_model(self):
        if self.input_shape is None:
            return None
        model = Sequential([
            Dense(64, activation='relu', input_shape=(self.input_shape,)),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    def calculate_job_match_score(self, resume_features, job_listing):
        score = 0
        # Skill match (up to 30 points)
        skill_match = len(set(resume_features['skills']) & set(job_listing['skills']))
        if job_listing['skills']:
            score += (skill_match / len(job_listing['skills'])) * 30
        # Experience match (up to 20 points)
        exp_range = job_listing['experience'].split('-')
        try:
            min_exp = int(exp_range[0])
        except:
            min_exp = 0
        resume_exp = resume_features['experience_years']
        score += max(0, 20 - abs(resume_exp - min_exp))
        # Domain match (20 points)
        domain_match = any(domain.lower() in job_listing['domain'].lower() for domain in resume_features['domains'])
        score += 20 if domain_match else 0
        # Project experience (20 points)
        score += 20 if resume_features['project_keywords'] else 0
        # Education bonus
        education_bonus = {'Bachelors': 5, 'Masters': 7, 'PhD': 10}
        if resume_features['education_level']:
            score += education_bonus.get(resume_features['education_level'], 0)
        # Certification bonus (up to 20 points)
        if job_listing['certifications_required']:
            matching_certs = set(resume_features['certifications']) & set(job_listing['certifications_required'])
            cert_bonus = (len(matching_certs) / len(job_listing['certifications_required'])) * 20
            score += cert_bonus
        return min(score, 100)

    def recommend_jobs(self, resume_features):
        job_recommendations = []
        for job in self.job_listings:
            match_score = self.calculate_job_match_score(resume_features, job)
            job_recommendations.append({
                'job_title': job['title'],
                'company': job['company'],
                'domain': job['domain'],
                'salary_range': job['salary_range'],
                'location': job['location'],
                'link': job['link'],
                'match_score': match_score
            })
        job_recommendations.sort(key=lambda x: x['match_score'], reverse=True)
        return job_recommendations

###############################################
# Part 7: Integrated Pipeline Main Function
###############################################
def main():
    # Step 1: Ask for Job Title (no location needed)
    job_title_filter = input("Enter job title to search for (e.g., Software Engineer): ").strip()
    if not job_title_filter:
        print("Job title cannot be empty. Exiting.")
        return

    # Step 2: Scrape Jobs from All Three Sources
    print("\nScraping Zscaler jobs...")
    zscaler_jobs = scrape_zscaler_jobs(job_title_filter)
    
    print("\nScraping Amazon jobs...")
    amazon_jobs = scrape_amazon_jobs(job_title_filter)
    
    print("\nScraping LinkedIn jobs...")
    linkedin_jobs = scrape_linkedin_jobs(job_title_filter, num_pages=3)
    
    combined_jobs = zscaler_jobs + amazon_jobs + linkedin_jobs
    if not combined_jobs:
        print("No jobs scraped from any source. Exiting.")
        return

    # Step 3: Enrich Raw Job Listings for Matching
    enriched_jobs = [enrich_job_listing(job) for job in combined_jobs]
    print("\nEnriched Job Listings:")
    for job in enriched_jobs:
        print(f"{job['title']} | {job['domain']} | {job['location']} | {job['link']}")

    # Step 4: Get Candidate Resume
    resume_choice = input("\nDo you want to upload a resume file? (y/n): ").strip().lower()
    resume_text = ""
    if resume_choice == 'y':
        file_path = input("Enter the path to your resume file (PDF, DOCX, TXT, PNG, JPG): ").strip()
        if os.path.exists(file_path):
            temp_recommender = BatchResumeRecommendationSystem(resume_folder="")
            resume_text = temp_recommender.extract_text_from_resume(file_path)
            if not resume_text:
                print("Could not extract text from the resume file. Exiting.")
                return
        else:
            print("File does not exist. Exiting.")
            return
    else:
        resume_text = input("Please paste your resume text:\n")
    
    if not resume_text.strip():
        print("Resume text is empty. Exiting.")
        return

    # Step 5: Initialize Recommendation System & Set Job Listings
    recommender = BatchResumeRecommendationSystem(resume_folder="")
    recommender.job_listings = enriched_jobs

    # Step 6: Extract Resume Features & Get Job Recommendations
    resume_features = recommender.extract_resume_features(resume_text)
    recommendations = recommender.recommend_jobs(resume_features)

    if recommendations:
        print("\nJob Recommendations based on your resume:")
        for rec in recommendations:
            print(f"Job Title: {rec['job_title']}")
            print(f"Company: {rec['company']}")
            print(f"Domain: {rec['domain']}")
            print(f"Location: {rec['location']}")
            print(f"Match Score: {rec['match_score']:.2f}")
            print(f"Link: {rec['link']}")
            print("-" * 50)
        # Save recommendations to CSV
        rec_df = pd.DataFrame(recommendations)
        rec_df.to_csv('job_recommendations.csv', index=False)
        print("Recommendations saved to 'job_recommendations.csv'")
    else:
        print("No matching jobs found.")

if __name__ == "__main__":
    main()


Enter job title to search for (e.g., Software Engineer):  Software Engineer



Scraping Zscaler jobs...
Scraping Zscaler jobs, filtering for 'Software Engineer'...
Skipping job 'Sr. Staff Software Engineer (C coding / Control Plane)' due to invalid link.
Found 13 jobs matching 'Software Engineer' from Zscaler.

Scraping Amazon jobs...
Scraping Amazon jobs, filtering for 'Software Engineer'...
No accept/reject popup found for Amazon.
Found 4 jobs matching 'Software Engineer' from Amazon.

Scraping LinkedIn jobs...
LinkedIn: Scraping page 1...
Found 60 job cards on page 1
LinkedIn: Scraped job 1/60: Software Engineer, AI Intern (Summer 2025) at Notion
LinkedIn: Scraped job 2/60: Software Engineer, AI Intern (Summer 2025) at Notion
LinkedIn: Scraped job 3/60: Full-Stack Software Engineer (New graduates: United States) at Wanderlog
LinkedIn: Scraped job 4/60: Software Engineer at Twitch
LinkedIn: Scraped job 5/60: Software Engineer at Lyft
LinkedIn: Scraped job 6/60: Software Engineer at Twitch
LinkedIn: Scraped job 7/60: Software Engineer at Twitch
LinkedIn: Scrape


Do you want to upload a resume file? (y/n):  y
Enter the path to your resume file (PDF, DOCX, TXT, PNG, JPG):  Software_Engineer_Resume.pdf



Job Recommendations based on your resume:
Job Title: Senior Software Engineer - Backend
Company: Zscaler
Domain: Engineering
Location: Hyderabad, IND
Match Score: 47.00
Link: https://boards.greenhouse.io/zscaler/jobs/4637689007
--------------------------------------------------
Job Title: Senior Software Engineer - DevTest
Company: Zscaler
Domain: Engineering
Location: Hyderabad, IND
Match Score: 47.00
Link: https://boards.greenhouse.io/zscaler/jobs/4637728007
--------------------------------------------------
Job Title: Senior Software Engineer - Full Stack
Company: Zscaler
Domain: Engineering
Location: Hyderabad, IND
Match Score: 47.00
Link: https://boards.greenhouse.io/zscaler/jobs/4637698007
--------------------------------------------------
Job Title: Staff Software Engineer - Golang
Company: Zscaler
Domain: Engineering
Location: San Jose, California, USA
Match Score: 47.00
Link: https://boards.greenhouse.io/zscaler/jobs/4624789007
------------------------------------------------

Starting scraping process...
LinkedIn: Scraping page 1...
LinkedIn: Error on page 1: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=134.0.6998.89)
Stacktrace:
0   chromedriver                        0x00000001051d5804 cxxbridge1$str$ptr + 2785964
1   chromedriver                        0x00000001051cdddc cxxbridge1$str$ptr + 2754692
2   chromedriver                        0x0000000104d21ea8 cxxbridge1$string$len + 92928
3   chromedriver                        0x0000000104cfba78 chromedriver + 129656
4   chromedriver                        0x0000000104d91498 cxxbridge1$string$len + 549104
5   chromedriver                        0x0000000104daa178 cxxbridge1$string$len + 650704
6   chromedriver                        0x0000000104d5d35c cxxbridge1$string$len + 335796
7   chromedriver                        0x000000010519acd4 cxxbridge1$str$ptr + 2545532
8   chromedriver                        0x000000010519dfa0 cxxbridg